In [2]:
import yaml
import pandas as pd
from numpy import triu
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
import nltk 
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
# Read in the processed data and the processed inbound dataset 
processed_df = pd.read_pickle("../data/processed/processed_v2.pkl")
processed_inbound_extra = processed_df["clean_inbound_text"]
processed_inbound_extra.head()

4     different people have given different answers ...
5     way to drop the ball on customer service so pi...
6     i want my amazon payments account closed dm me...
9     yeah this is crazy were less than a week away ...
10    how about you guys figure out my xbox one x pr...
Name: clean_inbound_text, dtype: object

In [5]:
# Reset the index. Set it properly in this iteration 
processed_df = processed_df.reset_index(drop=True)

In [6]:
# Check for some null values 
processed_df.isnull().sum()

inbound_text               0
author_id                  0
created_at                 0
outbound_text              0
response_tweet_id      60327
inbound_lang               0
inbound_hashtags           0
outbound_hashtags          0
clean_inbound_text         0
clean_outbound_text        0
outbound_tokens_pos        0
inbound_tokens_pos         0
dtype: int64

In [7]:
# Read in the intents back 
with open(r"../objects/intents_amazon_support.yml") as file:
    intents = yaml.load(file, Loader=yaml.FullLoader)

# Previewing
print(f'\nintents:\n{intents}')
print(f'\nprocessed:\n{processed_df.head()}')


intents:
{'track': ['tracking', 'order', 'shipment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details'], 'support': ['service'], 'quality': ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different'], 'discount': ['prime', 'product', 'offer', 'price', 'sale'], 'account': ['email', 'orders', 'details', 'bank', 'access']}

processed:
                                        inbound_text   author_id  \
0  @AmazonHelp 3 different people have given 3 di...  AmazonHelp   
1  Way to drop the ball on customer service @1158...  AmazonHelp   
2  @115823 I want my amazon payments account CLOS...  AmazonHelp   
3  @AmazonHelp @115826 Yeah this is crazy we’re l...  AmazonHelp   
4  @115828 How about you guys figure out my Xbox ...  AmazonHelp   

                  created_at  \
0  2017-10-31 23:28:00+00:00   
1  2017-10-31 22:29:00+00:00   
2  2017-10-31 22:28:34+00:00   
3  2017-11-01 12

## Tweet Collection with Doc2Vec 
I can use my Doc2Vec representation to find top 1000 Tweets most similar to a generalized intent version of a Tweet based on it's cosine similarity. 

Heuristic search refers to a search strategy that attempts to optimize a problem by iteratively improving the solution based on a given heuristic function or a cost measure. My cost measure is trying to get the closest cosine distances.

So I basically trained my doc2vec model with my training data, which is the `processed_inbound`. I can actually compute a vector based on my training data to vectorize that word.

In [10]:
def train_doc2vec(string_data, max_epochs, vec_size, alpha):
     
    # Tagging each of the documents with a unique ID
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(string_data)]
    
    # Instantiating my model 
    model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=1) # dm=1 means 'distributed memory' (PV-DM)
    
    # Building the vocabulary table
    model.build_vocab(tagged_data)
    
    for epoch in range(max_epochs): # Run for max_epochs
        print('iteration {0}'.format(epoch))    
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs) # This statement trains the model on the current epoch
        # Decreasing the learning rate
        model.alpha -= 0.0002
        # Fixing the learning rate, no decay
        model.min_alpha = model.alpha
        
    # Saving model
    model.save("../models/d2v.model")
    print("Model Saved")        
    
# Training
train_doc2vec(processed_inbound_extra, max_epochs=100, vec_size=20, alpha=0.025)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [ ]:
model = Doc2Vec()

In [1]:
# Creating a synthetic dataset - generating N Tweets resembling a mock tweet
# This will subsequently be merged with the existing inbound data for inclusion in the doc2vec training process

# Version 1 - will be improved in future iterations
ideal = {
            "order track": "@AmazonHelo Hi, could you provide an update on the order? Its been days since the product has moved from its last location ", # change intent to "order tracking"???
            "product inquiry": "@AmazonHelp Looking for more info on the product. Can you share details or direct me to a reliable source?", # product inquire??? 
            "return refund": "@AmazonHelp How can I start a return process? The item I received doesn't match the description.",
            "account management": "@AmazonHelp Hi, I am having trouble logging into my account. Can you help me reset my password?", 
            "promotion discount": "@AmazonHelp Are there any ongoing promotions or deals in the ongoing festive season? Looking to buy a few items.",
            "shipping": "@AmazonHelp Hi, My address has changed. Can you help me update the shipping address for my order?",
            "technical support": "@AmazonHelp Encountering errors during checkout. Can you help me troubleshoot the issue?",
            "payment issue": "@AmazonHelp My payment method isn't going through. Any suggestions on how to resolve this?",
            "general query": "@AmazonHelp Hi, I have a general question regarding the product. Can you help me with this?"
        }

In [ ]:
def add_extra(current_tokenized_data, extra_tweets):
    '''Adding extra tweets to the current tokenized data'''
    
    # Convert the extra tweets into a pandas Series
    extra_tweets = pd.Series(extra_tweets)

    # Convert the current tokenized data into a single string
    print('Converting to string...')
    string_processed_data = current_tokenized_data.progress_apply(" ".join)

    # Concatenate the extra tweets to the current data
    string_processed_data = pd.concat([string_processed_data, extra_tweets], axis = 0)

    # Tokenize the combined data
    tknzr = TweetTokenizer(strip_handles = True, reduce_len = True)
    print('Tokenizing...')
    tokenized_data = string_processed_data.progress_apply(tknzr.tokenize)

    return tokenized_data

# Add the extra tweets to the current data
processed_inbound_extra = add_extra(processed['Processed Inbound'], list(ideal.values()))

# Save the updated data to a pickle file
processed_inbound_extra.to_pickle('objects/processed_inbound_extra.pkl')

processed_inbound_extra

Here's is the catch. Have intent buckets inplace already in place. You would need to supply this to the below function to get your top N tweets corresponding to the current tweet


In [ ]:
# Dictionary mapping the intent to the row index 
intent_itags = {

}

def generate_intent(nsim, idx_tag): 
    '''Function that maps an index tag to an intent and returns nsim number of similar tweets'''
    sim_docs = model.docvecs.most_similar(idx_tag, topn = nsim)
    
    # Getting just the indexes 
    indexes = [int(i[0]) for i in sim_docs]
    
    # Actually seeing the top 1000 tweets similar to 0th tweet which seems to be about updates 
    # print(processed_inbound_extra[indexes])
    return indexes
    
# Create a dictionary mapping the intent to the row index of tweets
index_intents = {}
for intent, tag in intent_itags.items():
    print('Intent: ', intent)
    index_indents[intent] = generate_intent(1000, tag)
    print('\n')

In [ ]:
# Now map the index to each row of the preprocessed inbound data
preprocessed_inbound["intent"] = processed_inbounnd.index.map(index_intents)

In [ ]:
# Intent classification with Keras 
